# 11. EM 알고리즘과 GMM

기댓값-최대화(EM) 알고리즘과 가우시안 혼합 모델(GMM)을 구현합니다.

## 학습 목표
- EM 알고리즘의 E-step과 M-step 이해
- GMM의 원리와 적용

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(42)

## 1. 혼합 가우시안 데이터 생성

In [ ]:
# 3개의 가우시안 혼합
n_samples = 300

# 평균과 공분산
means = [torch.tensor([0.0, 0.0]), 
         torch.tensor([3.0, 3.0]), 
         torch.tensor([-2.0, 3.0])]

data = []
for i, mean in enumerate(means):
    samples = torch.randn(n_samples // 3, 2) * 0.5 + mean
    data.append(samples)

X = torch.cat(data, dim=0)

plt.figure(figsize=(8, 6))
plt.scatter(X[:, 0], X[:, 1], alpha=0.7)
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('Mixture of Gaussians')
plt.show()

## 2. GMM 적용

In [ ]:
from mlfs.probabilistic.em import GMM

In [ ]:
# GMM 학습
gmm = GMM(n_components=3, max_iters=100, random_state=42)
gmm.fit(X)

# 클러스터 예측
labels = gmm.predict(X)

print(f"Weights: {gmm.weights_}")
print(f"Means:\n{gmm.means_}")

In [ ]:
# 결과 시각화
plt.figure(figsize=(8, 6))
scatter = plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', alpha=0.7)

# GMM 중심 표시
means = gmm.means_
plt.scatter(means[:, 0], means[:, 1], c='red', marker='X', s=200, 
            edgecolors='white', linewidths=2, label='Centers')

plt.xlabel('x1')
plt.ylabel('x2')
plt.title('GMM Clustering')
plt.legend()
plt.colorbar(scatter, label='Cluster')
plt.show()

## 3. 소프트 할당 (Soft Clustering)

GMM은 각 점이 각 클러스터에 속할 확률을 제공합니다.

In [ ]:
# 확률 할당
proba = gmm.predict_proba(X)

print(f"Probability shape: {proba.shape}")
print(f"\nSample point probabilities:")
for i in range(5):
    print(f"  Point {i}: {proba[i].numpy().round(3)}")

## 요약

1. **E-step**: 현재 파라미터로 책임(responsibility) 계산
2. **M-step**: 책임을 기반으로 파라미터 업데이트
3. **GMM**: 소프트 클러스터링, 확률 모델
4. **K-Means와 비교**: GMM은 공분산도 모델링